In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [10]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [11]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="interpolate", line_search_cond="armijo")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1.1, c1=1e-4, tau=0.1, line_search_method="interpolate", line_search_cond="wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="interpolate", line_search_cond="strong-wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="interpolate", line_search_cond="goldstein")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0
1
, loss: 0.08934011310338974
epoch:  1
1
, loss: 0.06542723625898361
epoch:  2
1
, loss: 0.0473531149327755
epoch:  3
1
, loss: 0.037555139511823654
epoch:  4
1
, loss: 0.03503810986876488
epoch:  5
1
, loss: 0.03434830158948898
epoch:  6
1
, loss: 0.0341666117310524
epoch:  7
, loss: 0.034112948924303055
epoch:  8
, loss: 0.03409852832555771
epoch:  9
1
, loss: 0.0340900681912899
epoch:  10
, loss: 0.03405669704079628
epoch:  11
, loss: 0.03403681144118309
epoch:  12
, loss: 0.03401916101574898
epoch:  13
, loss: 0.03400436416268349
epoch:  14
1
, loss: 0.03399557247757912
epoch:  15
, loss: 0.033965252339839935
epoch:  16
, loss: 0.033946216106414795
epoch:  17
, loss: 0.03392781317234039
epoch:  18
, loss: 0.033911433070898056
epoch:  19
1
, loss: 0.033900149166584015
epoch:  20
, loss: 0.0338677279651165
epoch:  21
, loss: 0.03384474292397499
epoch:  22
, loss: 0.03382200002670288
epoch:  23
, loss: 0.03380100056529045
epoch:  24
, loss: 0.03378500044345856
epoch:  25
1


In [13]:
model.eval()
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = -33.48920814167194
Test metrics:  R2 = -29.75528042709307
